## Energy forecast - Train a simple linear model

## Import libraries

In [13]:
import tensorflow as tf
from model import model

## Verify read_dataset method to read in training data

In [10]:
seq_length = 8
n_forward = 5

TIME_SERIES_INPUT = "prices"

with tf.Session() as sess:
    fn = model.read_dataset(filename="data/train_{}_{}.csv".format(seq_length, n_forward),
                            mode=tf.estimator.ModeKeys.TRAIN,
                            seq_length=seq_length,
                            n_forward=n_forward,
                            batch_size=1)

    batch_features, batch_labels = fn()
    features, labels = sess.run([batch_features, batch_labels])
    print("try_out_input_function: features shape = {}".format(features[TIME_SERIES_INPUT]))
    print("try_out_input_function: labels shape = {}".format(labels))

try_out_input_function: features shape = [[63.29 62.1  59.49 57.63 57.1  57.97 53.87 53.  ]]
try_out_input_function: labels shape = [[56.57 60.22 60.47 54.71 49.  ]]


## Setup parameters

In [22]:
seq_length = 8
n_forward = 5
model = "linear"
out_dir_str = "trained/{}".format(model)

hparams = {}
hparams['model'] = model
hparams['train_data_path'] = "data/train_{}_{}.csv".format(seq_length, n_forward)
hparams['eval_data_path'] = "data/eval_{}_{}.csv".format(seq_length, n_forward)
hparams['seq_length'] = 8
hparams['n_forward'] = 5
hparams['learning_rate'] = 0.2
hparams['train_steps'] = 100
hparams['batch_size'] = 512
hparams['eval_delay_secs'] = 1
hparams['min_eval_frequency'] = 60

## Remove output folder so we can re-train the model

In [23]:
%%bash
OUTDIR=trained/linear
rm -rf $OUTDIR

## Train a linear model

In [24]:
from model import model

model.train_and_evaluate(out_dir_str, hparams)

INFO:tensorflow:Using config: {'_service': None, '_experimental_distribute': None, '_task_id': 0, '_tf_random_seed': None, '_save_checkpoints_secs': 60, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f16cf4866d8>, '_task_type': 'worker', '_evaluation_master': '', '_save_summary_steps': 100, '_log_step_count_steps': 100, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_global_id_in_cluster': 0, '_device_fn': None, '_protocol': None, '_train_distribute': None, '_model_dir': 'trained/linear', '_is_chief': True, '_num_worker_replicas': 1, '_eval_distribute': None, '_save_checkpoints_steps': None, '_master': ''}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evalua